In [1]:
# !pip install matplotlib
# !pip install pandas

# hastim@oregonstate.edu
# Mohan Krishna Hasti 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
#from numba import jit
#from tqdm.notebook import tqdm
from  multiprocessing import Pool

# High-performance oriented programming habits
## The good
+ reuse and preallocate variables
+ precompute data when possible; access when needed
+ vectorize whenever possible (we will practice together)
+ think about complementary approaches (sometimes easier to compute)
+ check variable data types (even in Python); only start this if result OK
## The bad
+ nested for-loops
+ branching; especially in nested loops
+ dynamically changing array sizes
## The ugly
+ np.vectorize is a for-loop 'wrapper'
+ not profiling your code
+ asking for a GPU without knowing why
## Please note
+ me=lazy programmer
+ same as you, I'm still learning Python3
+ and slow at typing (no jokes please while actively coding in class :-))

In [8]:
# not optimized; for your reference
#optimised by converting to np.array
sbox = np.array([
0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
])
# type(sbox)
k_array=np.array(range(256)) #array with 0-255 values
# print(k_array)
# print(sbox.index(43))

In [9]:
timings = np.loadtxt('timing_noisy.csv', delimiter=',')
timings.shape
plaintext = timings[:,range(16)]

In [10]:
np.shape(plaintext)

(1000000, 16)

In [56]:
from time import time
# t = time()
g1_msb1 = []
g0_msb0 = []
and_128 = np.empty(256, dtype = int)#array[0-255] filled with all 128
and_128.fill(128) 
for i in range(256):
  p_i = np.empty(256, dtype = int) #p_i byte, created 256 size p_i array 
  p_i.fill(i)
  # print(p_i)
  p_i = np.bitwise_xor(p_i,k_array)
  p_i = np.bitwise_and(sbox[p_i], and_128)
  p_i = np.floor_divide(p_i, 128)
  g1_msb1.append(p_i)
  # print(p_i)
  p_i = np.where((p_i==0)|(p_i==1), p_i^1, p_i) #complementing the values 1->0
  # print(p_i)
  g0_msb0.append(p_i)
g0_msb0 = np.array(g0_msb0) #converting g0_msb0 to np.array
g1_msb1 = np.array(g1_msb1) #converting g1_msb1 to np.array
# print(time() - t)
# print(g0_msb0)
# print(g1_msb1)

In [76]:
def crack_key(key_index):
  start = time()
  group1 = [np.zeros(256), np.zeros(256)] #creating group1 array with 2X256 = group1[0] - timings, group1[1] - 
  group0 = [np.zeros(256), np.zeros(256)] #creating group0 array with 2X256 = group0[0] - timings, group0[1] -
 
  #3b for i,row in enumerate(timings):
  for row in timings:
    key, timing = int(row[key_index]), row[-1] 
    
    group0[0] = np.add(group0[0], np.multiply (g0_msb0[key], timing)) #adding all the timings if MSB is 1
    group0[1] = np.add(group0[1], g0_msb0[key])  #adding all the key values 1 or 0
    group1[0] = np.add(group1[0], np.multiply (g1_msb1[key], timing))  #adding all the timings if MSB is 0
    group1[1] = np.add(group1[1], g1_msb1[key]) #adding all the key values 1 or 0
    #3b if i == 150000:
    #   break 

  group1_avg = np.divide(group1[0], group1[1]) #divide to find the average of group1 
  group0_avg = np.divide(group0[0], group0[1]) #divide to find the average of group0

  delta = np.subtract(group1_avg, group0_avg) #finding delta
  final_key = np.argmax(delta) #final key 
  #print index , KEY , time
  print(key_index, final_key, time() - start)  

  # generate graph
  %matplotlib inline
  print('Example')
  fig, ax = plt.subplots(figsize=(8, 4))
  ax.plot(delta, 'black')
  ax.set_xlabel('Samples')
  ax.set_ylabel('Value')
  plt.show()

#pooling multithreads 
with Pool(12) as p:
  all_start = time()
  p.map(crack_key, range(16)) #pool processes for 16 bytes  
  #Total execution time
  print("Total Time", time()- all_start)

#to run with single thread to get keys
# all_start = time()
# for i in range(16):
#   crack_key(i)
# print("Total Time", time()- all_start)

0 41 1.744248628616333
1 142 1.533984661102295
2 79 1.505387306213379
3 30 1.5168354511260986
4 183 1.5010223388671875
5 104 1.4784953594207764
6 193 1.4800307750701904
7 19 1.490391492843628
8 124 1.471010446548462
9 246 1.4732084274291992
10 189 1.4927635192871094
11 223 1.5359666347503662
12 236 1.519658088684082
13 119 1.4899511337280273
14 47 1.471174955368042
15 176 1.4680969715118408
Total Time 24.18728494644165


# Attack Recipe from Slides
+ Note: n = number of samples; corresponds to rows of .csv
+ Attacker guesses a value K as key candidate for the key
    + Create two groups: group1 and group0
    + for i = 1 to n compute Sbox(firstbyte(p_i) XOR k) – this will be our ‘hypothesis’
    + If MSB of result is 1, the corresponding timing t_i goes to group1, otherwise to group0
    + On average, the timings of group1 should be higher than of group0
    + Averaging helps to get this information and reduce noise
    + The difference between averages of two groups is assigned guessed key byte k
    + This step needs to be repeated for all key candidates 𝐾 = 0 . . . 255
+ The highest difference between the averages shows the most probable value candidate for the key

In [ ]:
# ## insert your code here ## perhaps start like this ...

# #for postion in range(16): # attack all bytes
#     #for row in range(15000): # read all/some rows of .csv
#         #for k in range(256): # guess our key

# # this approach will NOT work though in terms of time available
# for i in range(16):
#   for j in range(15000):
#     for k in range(256):

#       if sbox(np.logical_xor(i,k)): 

Please verify your result by plottng the timing differences. For each attacked byte, there will only be one clearly visible peak.

In [ ]:
# %matplotlib inline
# print('Example')
# fig, ax = plt.subplots(figsize=(8, 4))
# ax.plot(delta, 'black')

# ax.set_xlabel('Samples')
# ax.set_ylabel('Value')
# plt.show()